# PUMA: Promoter Unraveling through Machine-learning Algorithms
<img src="https://raw.githubusercontent.com/CarolusVitalis/PUMA/main/Images/PUMA_Logo.png" alt="PUMA Logo" width="200"/>


### _AI model to identify promoter sequences in existing databases_

### Introduction

Synthetic Biology aims to apply the engineering principles of abstraction, decoupling, and standardization to genetic engineering. In the context of synthetic biology, abstraction refers to the simplification of biological systems into fundamental, manageable components, allowing for a focus on critical interactions and functions while omitting less relevant details. Decoupling, on the other hand, involves the separation of interdependent systems within biology, enabling the independent design, modification, and analysis of each component. Together, these principles facilitate a modular and standardized approach to biological engineering, enhancing understanding, enabling innovative design, and promoting cross-disciplinary collaboration.

To improve standardization, synthetic biologists have developed notations and standardized DNA sequences for the different genetic parts, such as promoters, ribosome binding sites (RBSs), and coding sequences (CDSs), among others, which are then collected and deposited in repositories such as the iGEM Part Repository or SynBioHub. Although these repositories offer many different genetic parts and could serve as a good starting point, these parts tend to lack curation, which makes it difficult to identify, classify, and find ideal parts.

To overcome this challenge, an AI model was developed to recognize DNA sequences corresponding to Promoters from the iGEM Repository. This will make Synthetic Biology a more accessible field, greatly accelerating the rate of research and novel applications in a wide array of fields, from healthcare to agriculture, and even space colonization.

### Data Description

The model was trained with 4127 Promoters and 607 other parts, including RBSs, CDSs, Terminators, and Backbones from the 2023 iGEM Parts Distribution [1] and RegulonDB [2].

The iGEM data was scraped using the Part Retriever code [3]. A list of the necessary parts was provided and the code returned a CSV file with the part name and sequence. The parts were manually checked to ensure they were the parts that they claimed to be, this resulted in the deletion of two promoters from the promoters list provided.
The RegulonDB data was downloaded in CSV format using the front end and selecting just two fields, name, and sequence. After the files were downloaded, the sequences were standardized to use lowercase annotations.

[1] “iGEM Distribution.” Accessed: Feb. 19, 2024. [Online]. Available: https://technology.igem.org/distribution/handbook#h-whats-included-in-the-2023-distribution

[2]	“RegulonDB Browser.” Accessed: Apr. 09, 2024. [Online]. Available: https://regulondb.ccg.unam.mx/

[3] “Part_Retriever.” Accessed: Apr. 04, 2024. [Online] Available: https://github.com/CarolusVitalis/PUMA/blob/main/Part_Retriever.ipynb

### Methodology

On the first approach, a Gaussian Naive Bayes machine learning classification was used, achieving a training accuracy of 92.1%, but lacked generalizability to other species than e. coli and misidentified non-promoter genetic parts. Then, a Support Vector Classifier (SVC) model was used, resulting in a training accuracy of 92.3%.

### Results

The SVC model was applied to 2949 entries labeled as Promoters in the iGEM repository and identified 2019 entries as Promoters, 876 as CDS, 22 as RBS, and 32 as Terminators, a significant improvement from the Gaussian Naive Bayes model.

#### Dependencies
Here, we check if the dependencies are installed, and if they are not, ask the user if they want to install them

In [ ]:
import subprocess
import sys

# List of dependencies
dependencies = ["pandas", "scikit-learn", "seaborn", "plotnine"]

# Function to check if a module is installed
def is_module_installed(module_name):
    try:
        __import__(module_name)
        return True
    except ImportError:
        return False

# Function to install a module
def install_module(module_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", module_name])

# Check each dependency
for module in dependencies:
    if not is_module_installed(module):
        print(f"The module '{module}' is not installed.")
        answer = input(f"Do you want to install '{module}'? (yes/no): ")
        if answer.lower() == "yes":
            install_module(module)
            print(f"'{module}' has been installed.")
        else:
            print(f"'{module}' has not been installed.")
    else:
        print(f"The module '{module}' is already installed.")

We import the packages we need

In [ ]:
import seaborn as sns
import getpass
import requests
import re
import pandas as pd

### First Approach: Gaussian Naïve Bayes
This version uses local csv files from the iGEM Repository and the RegulonDB.

In [ ]:
import pandas as pd

#RegulonDB Files
pro_df = pd.read_csv('./RegulonDB_Files/promoters_Data.csv')
ter_df = pd.read_csv('./RegulonDB_Files/terminators_Data.csv')
gen_df = pd.read_csv('./RegulonDB_Files/genes_Data.csv')

print(pro_df)
print(ter_df)
print(gen_df)

# pro_df = pro_df.drop(pro_df.index[0])
# ter_df = ter_df.drop(ter_df.index[0])

pro_df['Element'] = "Promoter"
ter_df['Element'] = "Terminator"
gen_df['Element'] = "Gene"

print(pro_df)
print(ter_df)
print(gen_df)

#iGEM Repository Files
pro_2023 = pd.read_csv('./csv/Promoter_Part_Sequences.csv')
rbs_2023 = pd.read_csv('./csv/RBS_Part_Sequences.csv')
cds_2023 = pd.read_csv('./csv/CDS_Part_Sequences.csv')
ter_2023 = pd.read_csv('./csv/Terminator_Part_Sequences.csv')
bkb_2023 = pd.read_csv('./csv/Backbone_Part_Sequences.csv')

print(pro_2023)
print(rbs_2023)
print(cds_2023)
print(ter_2023)
print(bkb_2023)

# pro_2023 = pro_2023.drop(pro_2023.index[0])
# rbs_2023 = rbs_2023.drop(rbs_2023.index[0])
# cds_2023 = cds_2023.drop(cds_2023.index[0])
# ter_2023 = ter_2023.drop(ter_2023.index[0])
# bkb_2023 = bkb_2023.drop(bkb_2023.index[0])

pro_2023['Element'] = "Promoter"
rbs_2023['Element'] = "RBS"
cds_2023['Element'] = "CDS"
ter_2023['Element'] = "Terminator"
bkb_2023['Element'] = "Backbone"

In [ ]:
all_parts_df = pd.concat([pro_df, ter_df, pro_2023, rbs_2023, cds_2023, ter_2023, bkb_2023], ignore_index=True)
all_parts_df = all_parts_df.dropna(subset=['DNA Sequence'])
print(all_parts_df)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm

# Ask the user for the size of the k-mers
k_mer_size = int(input("Enter the size of the k-mers: "))

# Use CountVectorizer to initialize k-mer splitting 
# Use the user's input as the ngram_range parameter
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))

# apply the vectorizer to the concat. dataframe
X = vectorizer.fit_transform(all_parts_df['DNA Sequence']).toarray()
y = all_parts_df['Element']

In [ ]:
# these parameters are standard, but we can tweak if needed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
modelGaussian = GaussianNB()
modelGaussian.fit(X_train, y_train)

In [ ]:
predictions = modelGaussian.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


In [ ]:
X_new = vectorizer.transform(pro_2023['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_new = modelGaussian.predict(X_new)

# Add predictions to the new DataFrame
pro_2023['PredictedType'] = predictions_new

# Filter to identify sequences predicted as promoters
valid_promoters_df = pro_2023[pro_2023['PredictedType'] == 'Promoter']
print(len(pro_2023))
print(len(valid_promoters_df))
print("Valid Promoters")
print(valid_promoters_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_df = pro_2023[pro_2023['PredictedType'] != 'Promoter']
print(len(non_promoters_df))
print(non_promoters_df)
print(non_promoters_df[['Name', 'PredictedType']])

In [ ]:
training_nolabels = all_parts_df.drop('Element', axis=1)

X_old = vectorizer.transform(training_nolabels['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_old = modelGaussian.predict(X_old)

# Add predictions to the new DataFrame
training_nolabels['PredictedType'] = predictions_old

# Filter to identify sequences predicted as promoters
valid_promoters_old_df = training_nolabels[training_nolabels['PredictedType'] == 'Promoter']
print(len(all_parts_df))
print(len(valid_promoters_old_df))
print("Valid Promoters")
print(valid_promoters_old_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_old_df = training_nolabels[training_nolabels['PredictedType'] != 'Promoter']
print(len(non_promoters_old_df))
print(non_promoters_old_df)

import matplotlib.pyplot as plt
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit
import numpy as np
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)

common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 5),
    "cv": ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    "score_type": "both",
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Accuracy",
}

for ax_idx, estimator in enumerate([modelGaussian, modelSVC]):
    LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    handles, label = ax[ax_idx].get_legend_handles_labels()
    ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
    ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")

<img src="https://raw.githubusercontent.com/CarolusVitalis/PUMA/main/Images/Learning_Curves.png" alt="Learning Curves"/>

In [ ]:
igem_pro_file = open('./SBOL_Files/iGEM_Promoters_collection.xml').read()

# Search for DNA sequences using regular expression
igem_pro_dna_sequences = re.findall(r'<sbol:displayId>(.*?)_sequence</sbol:displayId>.*?<sbol:elements>(.*?)</sbol:elements>', igem_pro_file, re.DOTALL)

# Create a pandas dataframe
igem_pro_df = pd.DataFrame(igem_pro_dna_sequences, columns=['Name', 'DNA Sequence'])
print(len(igem_pro_df))
# Print the dataframe
#print(igem_pro_df)

# why is the first name of the df the name of the collection? 
# for now, will remove first row so that we have realistic names
igem_pro_df = igem_pro_df.drop(igem_pro_df.index[0])

# add in promoter label for these seqs
igem_pro_df['Element'] = "Promoter"

In [ ]:
igem_pro_nolabels = igem_pro_df.drop('Element', axis=1)
igem_pro_nolabels = igem_pro_nolabels.dropna(subset=['DNA Sequence'])


X_old = vectorizer.transform(igem_pro_nolabels['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_old = model.predict(X_old)

# Add predictions to the new DataFrame
igem_pro_nolabels['PredictedType'] = predictions_old

# Filter to identify sequences predicted as promoters
valid_igem_promoters_df = igem_pro_nolabels[igem_pro_nolabels['PredictedType'] == 'Promoter']
print(len(igem_pro_df))
print(len(valid_igem_promoters_df))
print("Valid Promoters")
print(valid_igem_promoters_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_igem_df = igem_pro_nolabels[igem_pro_nolabels['PredictedType'] != 'Promoter']
print(len(non_promoters_igem_df))
print(non_promoters_igem_df)

In [ ]:
import os

output_dir = "./csv"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, "igem_non_promoters.csv")
non_promoters_igem_df.to_csv(file_path, index=False)
print(f"Exported DataFrame as CSV: {file_path}")

### Second Approach: SVC
This version uses local csv files from the iGEM Repository and the RegulonDB.

In [ ]:
import pandas as pd

#RegulonDB Files
pro_df = pd.read_csv('./RegulonDB_Files/promoters_Data.csv')
ter_df = pd.read_csv('./RegulonDB_Files/terminators_Data.csv')
gen_df = pd.read_csv('./RegulonDB_Files/genes_Data.csv')

print(pro_df)
print(ter_df)
print(gen_df)

# pro_df = pro_df.drop(pro_df.index[0])
# ter_df = ter_df.drop(ter_df.index[0])

pro_df['Element'] = "Promoter"
ter_df['Element'] = "Terminator"
gen_df['Element'] = "Gene"

print(pro_df)
print(ter_df)
print(gen_df)

#iGEM Repository Files
pro_2023 = pd.read_csv('./csv/Promoter_Part_Sequences.csv')
rbs_2023 = pd.read_csv('./csv/RBS_Part_Sequences.csv')
cds_2023 = pd.read_csv('./csv/CDS_Part_Sequences.csv')
ter_2023 = pd.read_csv('./csv/Terminator_Part_Sequences.csv')
bkb_2023 = pd.read_csv('./csv/Backbone_Part_Sequences.csv')

print(pro_2023)
print(rbs_2023)
print(cds_2023)
print(ter_2023)
print(bkb_2023)

# pro_2023 = pro_2023.drop(pro_2023.index[0])
# rbs_2023 = rbs_2023.drop(rbs_2023.index[0])
# cds_2023 = cds_2023.drop(cds_2023.index[0])
# ter_2023 = ter_2023.drop(ter_2023.index[0])
# bkb_2023 = bkb_2023.drop(bkb_2023.index[0])

pro_2023['Element'] = "Promoter"
rbs_2023['Element'] = "RBS"
cds_2023['Element'] = "CDS"
ter_2023['Element'] = "Terminator"
bkb_2023['Element'] = "Backbone"

In [ ]:
all_parts_df = pd.concat([pro_df, ter_df, pro_2023, rbs_2023, cds_2023, ter_2023, bkb_2023], ignore_index=True)
all_parts_df = all_parts_df.dropna(subset=['DNA Sequence'])
print(all_parts_df)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm

# Ask the user for the size of the k-mers
k_mer_size = int(input("Enter the size of the k-mers: "))

# Use CountVectorizer to initialize k-mer splitting 
# Use the user's input as the ngram_range parameter
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))

# apply the vectorizer to the concat. dataframe
X = vectorizer.fit_transform(all_parts_df['DNA Sequence']).toarray()
y = all_parts_df['Element']

In [ ]:
# these parameters are standard, but we can tweak if needed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
modelSVC = svm.SVC()
modelSVC.fit(X_train, y_train)

In [ ]:
predictions = modelSVC.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


In [ ]:
X_new = vectorizer.transform(pro_2023['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_new = modelSVC.predict(X_new)

# Add predictions to the new DataFrame
pro_2023['PredictedType'] = predictions_new

# Filter to identify sequences predicted as promoters
valid_promoters_df = pro_2023[pro_2023['PredictedType'] == 'Promoter']
print(len(pro_2023))
print(len(valid_promoters_df))
print("Valid Promoters")
print(valid_promoters_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_df = pro_2023[pro_2023['PredictedType'] != 'Promoter']
print(len(non_promoters_df))
print(non_promoters_df)
print(non_promoters_df[['Name', 'PredictedType']])

In [ ]:
training_nolabels = all_parts_df.drop('Element', axis=1)

X_old = vectorizer.transform(training_nolabels['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_old = modelSVC.predict(X_old)

# Add predictions to the new DataFrame
training_nolabels['PredictedType'] = predictions_old

# Filter to identify sequences predicted as promoters
valid_promoters_old_df = training_nolabels[training_nolabels['PredictedType'] == 'Promoter']
print(len(all_parts_df))
print(len(valid_promoters_old_df))
print("Valid Promoters")
print(valid_promoters_old_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_old_df = training_nolabels[training_nolabels['PredictedType'] != 'Promoter']
print(len(non_promoters_old_df))
print(non_promoters_old_df)

import matplotlib.pyplot as plt
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit
import numpy as np
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)

common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 5),
    "cv": ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    "score_type": "both",
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Accuracy",
}

for ax_idx, estimator in enumerate([modelGaussian, modelSVC]):
    LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    handles, label = ax[ax_idx].get_legend_handles_labels()
    ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
    ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")

In [ ]:
igem_pro_file = open('./SBOL_Files/iGEM_Promoters_collection.xml').read()

# Search for DNA sequences using regular expression
igem_pro_dna_sequences = re.findall(r'<sbol:displayId>(.*?)_sequence</sbol:displayId>.*?<sbol:elements>(.*?)</sbol:elements>', igem_pro_file, re.DOTALL)

# Create a pandas dataframe
igem_pro_df = pd.DataFrame(igem_pro_dna_sequences, columns=['Name', 'DNA Sequence'])
print(len(igem_pro_df))
# Print the dataframe
#print(igem_pro_df)

# why is the first name of the df the name of the collection? 
# for now, will remove first row so that we have realistic names
igem_pro_df = igem_pro_df.drop(igem_pro_df.index[0])

# add in promoter label for these seqs
igem_pro_df['Element'] = "Promoter"

In [ ]:
igem_pro_nolabels = igem_pro_df.drop('Element', axis=1)
igem_pro_nolabels = igem_pro_nolabels.dropna(subset=['DNA Sequence'])


X_old = vectorizer.transform(igem_pro_nolabels['DNA Sequence']).toarray()

# Use the trained model to predict the sequence types
predictions_old = model.predict(X_old)

# Add predictions to the new DataFrame
igem_pro_nolabels['PredictedType'] = predictions_old

# Filter to identify sequences predicted as promoters
valid_igem_promoters_df = igem_pro_nolabels[igem_pro_nolabels['PredictedType'] == 'Promoter']
print(len(igem_pro_df))
print(len(valid_igem_promoters_df))
print("Valid Promoters")
print(valid_igem_promoters_df)

print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_igem_df = igem_pro_nolabels[igem_pro_nolabels['PredictedType'] != 'Promoter']
print(len(non_promoters_igem_df))
print(non_promoters_igem_df)

In [ ]:
import os

output_dir = "./csv"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_path = os.path.join(output_dir, "igem_non_promoters.csv")
non_promoters_igem_df.to_csv(file_path, index=False)
print(f"Exported DataFrame as CSV: {file_path}")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

all_parts_df


# Ask the user for the size of the k-mers
k_mer_size = int(input("Enter the size of the k-mers: "))

# Use CountVectorizer to initialize k-mer splitting 
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))

# Setup the SVM classifier
svm_model = SVC(kernel='linear')

# Create a pipeline that incorporates the vectorizer and SVM
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', svm_model)
])

# Parameters for Grid Search
param_grid = {'classifier__C': [0.1, 1, 10, 100], 'classifier__kernel': ['linear', 'rbf']}

# Setup nested cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Inner CV - hyperparameter tuning
clf = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv)

# Outer CV - model evaluation
nested_score = cross_val_score(clf, X=all_parts_df['DNA Sequence'], y=all_parts_df['Element'], cv=outer_cv)

print("Nested CV Accuracy: {:.2f} +/- {:.2f}".format(nested_score.mean(), nested_score.std()))


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from collections import Counter

all_parts_df
print(all_parts_df.shape[0])
print(all_parts_df.columns)
# User input for k-mer size
k_mer_size = int(input("Enter the size of the k-mers: "))

# Setup the CountVectorizer for k-mer splitting
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))

# Setup the SVM classifier
svm_model = SVC(kernel='linear')

# Create a pipeline with the vectorizer and SVM
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', svm_model)
])

# Parameters for Grid Search
param_grid = {'classifier__C': [.1, 1, 10, 100], 'classifier__kernel': ['linear', 'rbf']}

# Setup cross-validation
outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store results
best_params = []
accuracy_scores = []

for train_idx, test_idx in outer_cv.split(all_parts_df):
    X_train, X_test = all_parts_df.iloc[train_idx]['DNA Sequence'], all_parts_df.iloc[test_idx]['DNA Sequence']
    y_train, y_test = all_parts_df.iloc[train_idx]['Element'], all_parts_df.iloc[test_idx]['Element']
    
    # Inner CV - hyperparameter tuning
    clf = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)  # Inner cross-validation
    clf.fit(X_train, y_train)
    
    # Storing best parameters
    best_params.append(clf.best_params_)
    
    # Evaluate the classifier on the test set using the best found parameters
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Print the best parameters and accuracy for each outer fold
print("Best parameters and accuracy for each fold:")
for idx, (params, acc) in enumerate(zip(best_params, accuracy_scores)):
    print(f"Fold {idx+1}: Best Params: {params}, Accuracy: {acc:.4f}")

# Determine the most common best parameters
param_counts = Counter([frozenset(param.items()) for param in best_params])
most_common_params = param_counts.most_common(1)[0][0]
optimal_params = {k: v for k, v in most_common_params}

# Print the most common best parameters
print("Most common best parameters across all folds:", optimal_params)




In [ ]:
# Assuming best_params is a list of dictionaries obtained from GridSearchCV
from collections import Counter

# Calculate the most common best parameters
param_counts = Counter(tuple(sorted(d.items())) for d in best_params)
most_common_params, _ = param_counts.most_common(1)[0]

# Convert to dictionary and remove 'classifier__' prefix from keys
optimal_params = {k.split('__')[1]: v for k, v in most_common_params}

# Now you can create your final model with these parameters
final_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))),
    ('classifier', SVC(C=optimal_params['C'], kernel=optimal_params['kernel']))
])

# Train the final model with the full dataset
final_model.fit(all_parts_df['DNA Sequence'], all_parts_df['Element'])


# Now the final model is ready to make predictions on new data
all_promoters_good = pd.read_csv('./csv/All_Promoters_Part_Sequences.csv')

X_new = all_promoters_good['DNA Sequence']

# Use the trained model to predict the sequence types
predictions_new = final_model.predict(X_new)


# Add predictions to the new DataFrame
all_promoters_good['PredictedType'] = predictions_new

# Filter to identify sequences predicted as promoters
valid_promoters_df = all_promoters_good[all_promoters_good['PredictedType'] == 'Promoter']
print(len(all_promoters_good))
print(len(valid_promoters_df))
print("Valid Promoters")
print(valid_promoters_df)
valid_promoters_df.to_csv("./csv/valid_promoters.csv", index = False)


print("Non-Promoters")
# Filter to identify sequences predicted as non-promoters
non_promoters_df = all_promoters_good[all_promoters_good['PredictedType'] != 'Promoter']
print(len(non_promoters_df))
print(non_promoters_df)
print(non_promoters_df[['Name', 'PredictedType']])
non_promoters_df.to_csv("./csv/non_promoters.csv", index = False)



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import learning_curve, KFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

all_parts_df

# Set up the CountVectorizer for k-mer splitting
k_mer_size = 5  # Set k-mer size
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k_mer_size, k_mer_size))

# Set up the SVM classifier
svm_model = SVC(kernel='linear')

# Create a pipeline with the vectorizer and SVM
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', svm_model)
])

# Prepare training data
X, y = all_parts_df['DNA Sequence'], all_parts_df['Element']

# Generate learning curve data
train_sizes, train_scores, test_scores = learning_curve(
    estimator=pipeline,
    X=X,
    y=y,
    train_sizes=np.linspace(.1, 1, 100),
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',
    n_jobs=-1  # Use all processors
)

# Calculate mean and standard deviation for train and test scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Train score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.15)

plt.plot(train_sizes, test_mean, label='Cross-validation score', color='green', marker='o')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='green', alpha=0.15)

plt.title('Learning Curve for SVM Classifier')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy Score')
plt.legend(loc='best')
plt.grid(True)
plt.show()


<img src="https://raw.githubusercontent.com/CarolusVitalis/PUMA/main/Images/SVC_Model.png" alt="SVC Model"/>

### Discussion
The model showed the importance of curating existing databases, especially the iGEM student entries in order to ensure they can be used by scientists without worrying about whether the parts are correctly labeled or not.

Tools such as PUMA can help in this regard, ensuring that easily accessible data is also trust worthy. In its current form, PUMA shows impressive results when identifying what is or is not a promoter, but it is not yet able to correctly identify other parts due to the lack of training data for other parts. That opens a door for further development in identification of other parts and even composite sequences.